In [2]:
from model import *
from Vocabulary import vocabulary
import pickle
from preprocessing import normalizeString

In [78]:
#loading app vocabulary
vocab_path = "training_runs/chatbot_1/vocab.pkl"
with open(vocab_path, "rb") as f:
    vocab = pickle.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#initializing models
embedding = nn.Embedding(vocab.num_words, 512)
encoder = Encoder(512,embedding,2,0.1)
decoder = Decoder("dot", embedding, 512, vocab.num_words, 2, 0.1)
#loading appropriate weights
missing, unexpected = encoder.load_state_dict(torch.load("training_runs/chatbot_1/train_10/encoder.pth"))
decoder.load_state_dict(torch.load("training_runs/chatbot_2/train_10/decoder.pth"))
# embedding.load_state_dict(torch.load("training_runs/chatbot_2/train_50/embedding.pth"))

encoder.eval()
decoder.eval()

encoder = encoder.to(device)
decoder = decoder.to(device)

In [79]:
s = "hello"

In [80]:
sentance = normalizeString(s)

In [81]:
indexes_batch = [vocab.indexfromSentance(sentance)]

In [82]:
indexes_batch

[[855, 2]]

In [87]:
lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # make (batch, seq_len) -> (seq_len, batch)
input_batch = torch.LongTensor(indexes_batch).transpose(0,1).to(device)

In [88]:
lengths

tensor([2])

In [89]:
input_batch.shape

torch.Size([2, 1])

In [90]:
encoder_output, encoder_hidden = encoder(input_batch, lengths)

In [91]:
decoder_hidden = encoder_hidden[:decoder.n_layers]

In [92]:
decoder_input = torch.ones(1,1, device=device, dtype=torch.long) * 1

In [93]:
#Inititalize tensors to append decoded words to
all_tokens = torch.zeros(0, device=device, dtype=torch.long)
all_scores = torch.zeros(0, device=device)

In [94]:
decoder_input.device

device(type='cuda', index=0)

In [95]:
for _ in range(11):
    decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_output)
    decoder_input = torch.multinomial(decoder_output,1)
    all_tokens = torch.cat((all_tokens, decoder_input), dim=0)

In [96]:
all_tokens

tensor([[  34],
        [ 872],
        [ 102],
        [  11],
        [  18],
        [   8],
        [2184],
        [1724],
        [  11],
        [   2],
        [   0]], device='cuda:0')

In [97]:
decoded_words = [vocab.index2word[token.item()] for token in all_tokens]

In [98]:
decoded_words

['i', 'couldn', 't', '.', 'have', 'a', 'flower', 'feed', '.', 'EOS', 'PAD']

In [47]:
decoder_output.shapea

torch.Size([1, 9030])

In [48]:
torch.max(decoder_output, dim=1)

torch.return_types.max(
values=tensor([0.0930], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([34], device='cuda:0'))

In [49]:
vocab.index2word[34]

'i'

In [50]:
sum(decoder_output[0])

tensor(1.0000, device='cuda:0', grad_fn=<AddBackward0>)

In [53]:
decoder_input = torch.multinomial(decoder_output,1)

In [52]:
vocab.index2word[17]

'you'

In [54]:
decoder_input.shape

torch.Size([1, 1])